# Demographic implications of cloning in marine invertebrate larvae 
This is a graphical interface for using the `clone_modelND` (for Non-Dimensional) model of low-Reynolds number hydrodynamics. The `clone_modelND` model calculates the size distribution of a cohort of marine invertebrate larvae over time. This cohort represents a group of larvae beginning development at the same time and at the same size. For example, the cohort could represent larvae from the eggs of a single mother, or larvae from the eggs of a local population of mothers that spawned at a given time and place.

The cohort of larvae undergoes growth and development, but also is reduced in numbers by mortality. Like most larval life history models in the literature, **size is assumed in the** `clone_modelND` **model to be a *proxy* for developmental state**. This means that the model does not represent developmental state directly. Instead, it assumes that developmental state is closely coupled to size. For example, a larva is assumed to be competent to settle when it reaches a specific size. This approach means that the model does not exactly describe many details of morphology and life history associated with any given species. However, the more general results of this modeling approach may approximately describe a great many different species.

In "normal" development (without cloning), the cohort of larvae always decreases in population due to mortality, because there is no mechanism for adding new larvae to the cohort. In a cloning species, one larva can divide into two genetically identical daughter larvae. This means that, in cloning larvae, the number of larvae in a cohort can either increase or decrease.  

However, while cloning produces two larvae out of one, it does not provide any additional tissue or nutritional resources to those larvae. This means that a larva which clones produces two "daughter" larvae are smaller. The model assumes that the daughter larvae divide the resources from the original larva. Hence, **the sizes of the two daughters add up to the size of the original larva.**



In [ ]:
# Load modules and set graphics environment
from math import *
import numpy as np
from matplotlib import pyplot as plt
plt.ion();
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from Cloning.clone_modelND import Params, ClonePDE

In [ ]:
# Instantiate a Params object and a GUI to modify parameters. 
params=Params(auto_plot=True)
# Restrict to a single environment, to simplify the interface
#save_all=False,s_egg=0.1,s_0=0.75,s_1=0.25,s_2=1.,
#                 Smax=1.25,s_j=1.,N0=1.,n_env=1,
#                 t_env_cycle=1.,env_cycle_ratios=np.array([1.]),g_pars=[{'g0':1.}],
#                 m_pars=[{'m0':0.15}],alpha=-0.75,c_0 = 0.5,f_2 = 1.,C = None,
#                 nruns=1,start_time=0.,end_time=10.,dt=0.05,max_step=0.005,ns=4*32,
#                 abserr=1.e-12,relerr=1.e-10,auto_plot=False
s_egg=widgets.FloatText(value=0.1,width=10,description = r"$s_{egg}$")
s_0=widgets.FloatText(value=0.75,description = r"$s_0$")
s_1=widgets.FloatText(value=0.25,description = r"$s_1$")
s_2=widgets.FloatText(value=1.0,description = r"$s_2$")
Smax=widgets.FloatText(value=1.25,description = r"$s_{max}$")
Tmax=widgets.FloatText(value=10.,description = r"$T_{max}$")
g0=widgets.FloatText(value=1.,description = r"$g_0$")
m0=widgets.FloatText(value=0.15,description = r"$m_0$")
alpha=widgets.FloatText(value=-0.75,description = r"$\alpha$")
c_0=widgets.FloatText(value=0.5,description = r"$c_0$")
f_2=widgets.FloatText(value=1.,description = r"$f_2$")

h = '30px'
L_s_egg = widgets.Label(value='Egg size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="97%",height=h))
L_s_0 = widgets.Label(value='Size at cloning',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="95%",height=h))
L_s_1 = widgets.Label(value='Clone size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="95%",height=h))
L_s_2 = widgets.Label(value='Size at metamorphosis',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="97%",height=h))
L_Smax = widgets.Label(value='Plot limit',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="90%",height=h))
uiL1 = widgets.VBox([L_s_egg,L_s_0,L_s_1,L_s_2,L_Smax])

L_g0 = widgets.Label(value='Growth rate',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="97%",height=h))
L_m0 = widgets.Label(value='Mortality rate',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_alpha = widgets.Label(value='Mort. selectivity',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_c_0 = widgets.Label(value='Cloning probability',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_f_2 = widgets.Label(value='Metamorphosis probability',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="97%",height=h))
L_Tmax = widgets.Label(value='End time',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="90%",height=h))
uiL2 = widgets.VBox([L_g0,L_m0,L_alpha,L_c_0,L_f_2,L_Tmax])


LabelObj = widgets.Label(value='Object',layout=widgets.Layout(display="flex", 
                                        justify_content="center",width="100%"))
LabelObj1 = widgets.Label(value='Object')
Space = widgets.Label(value=' ',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="center", 
                                               width="100%"))
LabelModel = widgets.Label(value='Model',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="flex-end", 
                                               width="150%"))
ui0 = widgets.VBox([LabelObj,LabelObj1,Space,LabelModel])
ui1 = widgets.VBox([s_egg,s_0,s_1,s_2,Smax])
ui2 = widgets.VBox([g0,m0,alpha,c_0,f_2,Tmax])
ui3 = widgets.HBox([uiL1,ui1,ui2,uiL2])

def setParams(s_egg,s_0,s_1,s_2,Smax,g0,m0,alpha,c_0,f_2,Tmax):
    params=Params(auto_plot=True)
    params.s_egg = s_egg
    params.s_0 = s_0
    params.s_1 = s_1
    params.s_2 = s_2
    params.Smax = Smax
    params.g_pars = [{'g0':g0}]
    params.m_pars = [{'m0':m0}]
    params.alpha = alpha
    params.c_0 = c_0
    params.f_2 = f_2
    params.end_time = Tmax
    result=ClonePDE(params)
    
out = widgets.interactive_output(setParams,{'s_egg':s_egg,'s_0':s_0,'s_1':s_1,'s_2':s_2,'Smax':Smax,
                                             'g0':g0,'m0':m0,'alpha':alpha,'c_0':c_0,'f_2':f_2,'Tmax':Tmax})
display(ui3,out)